# Copyright © 2022 Gurobi Optimization, LLC

# Integrate a neural network in a Gurobi model

We take the model from JANOS example:

$
\begin{align}
&\max \sum y_i \\
&\text{subject to:}\\
&\sum x_i \le 100,\\
&y_i = g(x_i, Xfix),\\
& 0 \le x \le 2.5.
\end{align}
$

Where, $Xknown$ is a vector of fixed features. And $g$ is a neural network computed using scikit-learn that predicts the probabilty that a student will join.

In [ ]:
import sys

import gurobipy as gp
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
sys.path.append('../../src/')
from ml2gurobi.sklearn2gurobi import Pipe2Gurobi

### Do our Tree regression

In [ ]:
# Retrieve historical data used to do the regression
historical_data = pd.read_csv(
    'data/college_student_enroll-s1-1.csv', index_col=0)

# Classify our features between the ones that are fixed and the ones
# that will be part of the optimization problem
known_features = ['SAT', 'GPA']
dec_features = ['scholarship']
features = known_features + dec_features

# The target for training
target = 'enroll'

historical_data = historical_data[features + [target]]

In [ ]:
# Run our regression

X = historical_data.loc[:, features]
Y = historical_data.loc[:, 'enroll']
scaler = StandardScaler()

layers = [10] * 2
regression = DecisionTreeRegressor()
pipe = make_pipeline(scaler, regression)
pipe.fit(X=X, y=Y)

### Do the optimization model

In [ ]:
# Retrieve new data used for the optimization
studentsdata = pd.read_csv('data/admissions500.csv', index_col=0)
studentsdata = studentsdata[known_features]
# Check that features are in identical order

assert (studentsdata.columns.get_indexer(known_features)
        == historical_data.columns.get_indexer(known_features)).all()

nstudents = studentsdata.shape[0]

In [ ]:
# Start with classical part of the model
m = gp.Model()

knownidx = historical_data.columns.get_indexer(known_features)
decidx = historical_data.columns.get_indexer(dec_features)

lb = np.zeros((nstudents, len(features)))
ub = np.ones((nstudents, len(features))) * 2.5
lb[:, knownidx] = studentsdata.loc[:, known_features]
ub[:, knownidx] = studentsdata.loc[:, known_features]

x = m.addMVar(lb.shape, lb=lb, ub=ub, name='x')
y = m.addMVar(nstudents, lb=-gp.GRB.INFINITY, name='y')

m.setObjective(y.sum(), gp.GRB.MAXIMIZE)
m.addConstr(x[:, decidx].sum() <= 0.2 * nstudents)

# create transforms to turn scikit-learn pipeline into Gurobi constraints
pipe2gurobi = Pipe2Gurobi(pipe, m)

# Add constraint to predict value of y using kwnown and to compute features
pipe2gurobi.predict(X=x, y=y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 20
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

In [ ]:
m.write('toto.lp')

### Look at the solution

In [ ]:
# This is what we predicted
plt.scatter(x.X[:, decidx], y.X)

In [ ]:
# This is the historical data
plt.scatter(X.iloc[:, decidx], pipe.predict(X))

In [ ]:
# Proportion of students offered a scholarship
print("In historical data {:.4}% students offered a scholarship".format(
    100 * ((X.iloc[:, decidx] > 0).sum()[0] / len(X.iloc[:, decidx]))))
print("In our solution {:.4}% students offered a scholarship".format(
    100 * sum(x.X[:, decidx] > 0)[0] / nstudents))

In [ ]:
bottom10 = historical_data.sort_values(by=['GPA'])[:2000]

top10 = historical_data.sort_values(by=['GPA'])[-2000:]

top10.enroll.sum()

print("In historical data")
print("Among top 10% students: {}% were offered a scholarship and {}% joined".format(
      100 * (top10.scholarship > 0).sum() / top10.shape[0],
      100 * (top10.enroll.sum() / top10.shape[0])))
print("Among bottom 10% students: {}% were offered a scholarship and {}% joined".format(
      100 * (bottom10.scholarship > 0).sum() / bottom10.shape[0],
      100 * (bottom10.enroll.sum() / bottom10.shape[0])))

Xpredicted = pd.concat([studentsdata,
                        pd.DataFrame(x.X[:, decidx], columns=['scholarship'],
                                     index=studentsdata.index),
                        pd.DataFrame(y.X, columns=['enroll'], index=studentsdata.index)],
                       axis=1)
top10 = Xpredicted.sort_values(by='GPA')[-50:]
bottom10 = Xpredicted.sort_values(by='GPA')[:50]

print("In predicted data")
print("Among top 10% students: {:.4}% were offered a scholarship and {:.4}% joined".format(
      100 * (top10.scholarship > 0).sum() / top10.shape[0],
      100 * (top10.enroll.sum() / top10.shape[0])))
print("Among bottom 10% students: {:.4}% were offered a scholarship and {:.4}% joined".format(
      100 * (bottom10.scholarship > 0).sum() / bottom10.shape[0],
      100 * (bottom10.enroll.sum() / bottom10.shape[0])))

Copyright © 2022 Gurobi Optimization, LLC